# Leonardo's Bobbin Winder: Automation in Textile

> *"This is the way to wind silk..."*  
> — Leonardo da Vinci

## Introduction
Leonardo designed an automatic bobbin winder to improve the efficiency of the textile industry. A key feature was a **cam mechanism** that moved the thread guide back and forth, ensuring the thread was wound evenly across the spool.

In this notebook, we analyze:
1.  **Cam Profiles**: How different cam shapes affect winding patterns.
2.  **Uniformity**: Ensuring the spool remains flat and even.
3.  **Tension**: Maintaining constant thread tension.

---

In [ ]:
# Install the davinci-codex library
!pip install -q git+https://github.com/Shannon-Labs/davinci-codex.git

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Set "Brutalist Academic" plotting style
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'text.color': 'black',
    'axes.labelcolor': 'black',
    'xtick.color': 'black',
    'ytick.color': 'black'
})


## The Physics Model

### 1. Cam Profiles
- **Heart Cam**: Linear motion, constant velocity traverse. Good for uniformity but causes shock at reversal.
- **Spiral Cam**: Smooth motion, but non-linear traverse speed.
- **Compound Cam**: Leonardo's optimization for smooth reversal and linear traverse.

### 2. Winding Uniformity
The thread density $\rho(x)$ across the spool width $W$ depends on the traverse velocity $v_{traverse}$:
$$ \rho(x) \propto \frac{1}{v_{traverse}(x)} $$
Constant velocity is ideal for uniform density.

In [ ]:
def generate_cam(cam_type, lift):
    theta = np.linspace(0, 2*np.pi, 360)
    
    if cam_type == 'Heart':
        # Linear rise and fall
        # 0 to pi: rise
        rise = (lift / np.pi) * theta[:180]
        # pi to 2pi: fall
        fall = lift - (lift / np.pi) * (theta[180:] - np.pi)
        displacement = np.concatenate([rise, fall])
        
    elif cam_type == 'Spiral':
        # Sinusoidal (Archimedean spiral approximation for smooth motion)
        displacement = 0.5 * lift * (1 - np.cos(theta))
        
    elif cam_type == 'Compound':
        # Hybrid: Linear middle, smoothed ends
        # Simplified approximation
        displacement = np.zeros_like(theta)
        for i, t in enumerate(theta):
            phase = t % (2*np.pi)
            if phase < np.pi:
                # Rise
                displacement[i] = lift * (phase / np.pi)
                # Smooth start/end
                displacement[i] += 0.1 * lift * np.sin(2 * phase)
            else:
                # Fall
                displacement[i] = lift * (1 - (phase - np.pi) / np.pi)
                displacement[i] -= 0.1 * lift * np.sin(2 * (phase - np.pi))
                
    return theta, displacement

def plot_winder(cam_type, speed_rpm):
    lift = 10.0 # cm
    theta, displacement = generate_cam(cam_type, lift)
    
    # Calculate Velocity (Traverse Speed)
    # omega = rpm * 2pi / 60
    omega = speed_rpm * 2 * np.pi / 60.0
    # v = d(disp)/dt = d(disp)/dtheta * dtheta/dt = d(disp)/dtheta * omega
    velocity = np.gradient(displacement, theta) * omega
    
    # Calculate Density (Inverse of speed)
    # Avoid divide by zero
    density = 1.0 / (np.abs(velocity) + 0.1)
    density = density / np.mean(density) # Normalize
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot 1: Cam Profile
    ax1.plot(np.degrees(theta), displacement, 'b-', linewidth=2)
    ax1.set_title(f'{cam_type} Cam Profile')
    ax1.set_xlabel('Cam Angle (deg)')
    ax1.set_ylabel('Traverse Position (cm)')
    ax1.grid(True)
    
    # Plot 2: Winding Density
    # Map displacement (spool width) to density
    # We sort by displacement to show density across the spool width
    sorted_indices = np.argsort(displacement)
    ax2.plot(displacement[sorted_indices], density[sorted_indices], 'r-', linewidth=2)
    ax2.set_title('Winding Density Uniformity')
    ax2.set_xlabel('Spool Position (cm)')
    ax2.set_ylabel('Relative Thread Density')
    ax2.set_ylim(0, 2.0)
    ax2.axhline(1.0, color='k', linestyle='--', label='Ideal Uniformity')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    uniformity_score = 1.0 / np.std(density)
    print(f"Uniformity Score: {uniformity_score:.2f} (Higher is better)")
    if cam_type == 'Heart':
        print("Note: Heart cam gives uniform winding but has sharp velocity changes.")
    elif cam_type == 'Spiral':
        print("Note: Spiral cam is smooth but causes uneven winding (more thread at ends).")

interact(plot_winder, 
         cam_type=widgets.Dropdown(options=['Heart', 'Spiral', 'Compound'], value='Heart', description='Cam Type'),
         speed_rpm=widgets.IntSlider(min=10, max=100, step=10, value=60, description='Speed (RPM)'));

## Conclusion

The **Heart Cam** provides the most uniform winding because it moves the thread guide at a constant speed. However, the sharp reversal at the ends can break delicate threads. Leonardo's **Compound Cam** designs attempted to find a compromise: maintaining near-constant speed for most of the stroke while smoothing the reversal to prevent breakage.